In [16]:
import requests
import json
import time

def scrape_all_card_lists():
    all_summaries = []
    base_url = "https://api.card-gorilla.com:8080/v1/cards"
    
    # 제공해주신 브라우저 헤더 정보 반영
    headers = {
        "authority": "api.card-gorilla.com:8080",
        "accept": "application/json, text/plain, */*",
        "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "origin": "https://www.card-gorilla.com",
        "referer": "https://www.card-gorilla.com/",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"
    }

    print("=== [1단계] 모든 카드 목록 요약 수집 시작 ===")

    # total이 1075개이므로 10개씩 호출 시 약 108페이지까지 존재
    # 전수 조사를 위해 범위를 110으로 설정
    for page in range(1, 111):
        params = {
            "p": page,
            "perPage": 10,
            "cate": "CRD"
        }

        try:
            response = requests.get(base_url, headers=headers, params=params, timeout=10)
            
            if response.status_code == 200:
                json_data = response.json()
                cards = json_data.get('data', [])
                
                if not cards:
                    print(f"\n{page}페이지: 수집할 데이터가 더 이상 없습니다.")
                    break
                
                # 핵심 식별 정보만 추출하여 저장 공간 최적화
                for card in cards:
                    summary = {
                        "idx": card.get("idx"),
                        "name": card.get("name"),
                        "corp": card.get("corp_txt"),
                        "type": card.get("cate_txt"),
                        "min_perf": card.get("pre_month_money")
                    }
                    all_summaries.append(summary)
                
                print(f"진행 중: {page}페이지 완료 (누적 {len(all_summaries)}개 수집)")
                
                # 서버 부하 방지를 위한 간격 (안정적 수집을 위해 필수)
                time.sleep(0.5)
            else:
                print(f"\n오류: {page}페이지 호출 실패 (상태 코드: {response.status_code})")
                break

        except Exception as e:
            print(f"\n시스템 예외 발생: {e}")
            break

    # 결과 저장
    if all_summaries:
        file_name = "all_cards_summary.json"
        with open(file_name, "w", encoding="utf-8") as f:
            json.dump(all_summaries, f, ensure_ascii=False, indent=4)
        print(f"\n[성공] 총 {len(all_summaries)}개의 카드 목록을 '{file_name}'에 저장했습니다.")
    else:
        print("\n[실패] 수집된 데이터가 없습니다.")

if __name__ == "__main__":
    scrape_all_card_lists()

=== [1단계] 모든 카드 목록 요약 수집 시작 ===
진행 중: 1페이지 완료 (누적 10개 수집)
진행 중: 2페이지 완료 (누적 20개 수집)
진행 중: 3페이지 완료 (누적 30개 수집)
진행 중: 4페이지 완료 (누적 40개 수집)
진행 중: 5페이지 완료 (누적 50개 수집)
진행 중: 6페이지 완료 (누적 60개 수집)
진행 중: 7페이지 완료 (누적 70개 수집)
진행 중: 8페이지 완료 (누적 80개 수집)
진행 중: 9페이지 완료 (누적 90개 수집)
진행 중: 10페이지 완료 (누적 100개 수집)
진행 중: 11페이지 완료 (누적 110개 수집)
진행 중: 12페이지 완료 (누적 120개 수집)
진행 중: 13페이지 완료 (누적 130개 수집)
진행 중: 14페이지 완료 (누적 140개 수집)
진행 중: 15페이지 완료 (누적 150개 수집)
진행 중: 16페이지 완료 (누적 160개 수집)
진행 중: 17페이지 완료 (누적 170개 수집)
진행 중: 18페이지 완료 (누적 180개 수집)
진행 중: 19페이지 완료 (누적 190개 수집)
진행 중: 20페이지 완료 (누적 200개 수집)
진행 중: 21페이지 완료 (누적 210개 수집)
진행 중: 22페이지 완료 (누적 220개 수집)
진행 중: 23페이지 완료 (누적 230개 수집)
진행 중: 24페이지 완료 (누적 240개 수집)
진행 중: 25페이지 완료 (누적 250개 수집)
진행 중: 26페이지 완료 (누적 260개 수집)
진행 중: 27페이지 완료 (누적 270개 수집)
진행 중: 28페이지 완료 (누적 280개 수집)
진행 중: 29페이지 완료 (누적 290개 수집)


KeyboardInterrupt: 

In [20]:
import requests
import json

def get_card_main_info(idx=13):
    url = f"https://api.card-gorilla.com:8080/v1/cards/{idx}"
    headers = {
        "authority": "api.card-gorilla.com:8080",
        "accept": "application/json, text/plain, */*",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            data = response.json()
            
            # 1. 카드 메인 설명 (이벤트 타이틀이나 카드사 한줄 요약)
            # 'event' 내의 'card_detail_text'가 사용자가 가장 먼저 보게 되는 메인 문구입니다.
            main_desc = data.get('event', {}).get('card_detail_text', "상세 설명 없음")
            
            # 2. 주요 혜택 리스트 추출
            # 'top_benefit' 리스트 안에 카테고리별 혜택과 태그(할인율 등)가 들어있습니다.
            benefits = []
            top_benefits = data.get('top_benefit', [])
            
            for b in top_benefits:
                title = b.get('title')
                tags = b.get('tags', []) # ['온라인 쇼핑', '10%', '청구할인'] 등
                
                if title: # 빈 값 제외
                    benefit_str = f"[{title}] {' / '.join(tags)}"
                    benefits.append(benefit_str)

            # 3. 최종 결과 구조화
            result = {
                "idx": data.get("idx"),
                "name": data.get("name"),
                "main_description": main_desc,
                "key_benefits": benefits
            }
            
            print(f"=== {result['name']} 핵심 정보 추출 결과 ===")
            print(json.dumps(result, ensure_ascii=False, indent=4))
            return result

    except Exception as e:
        print(f"에러 발생: {e}")

if __name__ == "__main__":
    get_card_main_info(13)

=== 신한카드 Mr.Life 핵심 정보 추출 결과 ===
{
    "idx": 13,
    "name": "신한카드 Mr.Life",
    "main_description": "최대 52만원 캐시백",
    "key_benefits": [
        "[공과금] 공과금 / 10% / 할인",
        "[마트/편의점] 마트,편의점 / 10% / 할인",
        "[푸드] 식음료 / 10% / 할인"
    ]
}
